# Try out different time series classification algorithms

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from datasets import Dataset
from param import *

datalist = ParamDir().data_path_list

In [2]:
data_dir = datalist[2]
dataset = Dataset(data_dir, 1.0, False)

(X_train, y_train), (X_test, y_test) = dataset.load_all_data(10, .8)

# Classification with Unequal Length Series

In [3]:
# from sktime.registry import all_estimators

# # search for all classifiers which can handle unequal length data. This may give some
# # UserWarnings if soft dependencies are not installed.
# all_estimators(
#     filter_tags={"capability:unequal_length": True}, estimator_types="classifier"
# )

from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.transformations.panel.padder import PaddingTransformer

padded_clf = PaddingTransformer() * RandomIntervalClassifier(n_intervals=4)
padded_clf.fit(X_train, y_train)
y_pred = padded_clf.predict(X_test)

/usr/local/lib/python3.8/site-packages/sktime/utils/validation/_dependencies.py:143: UserWarning: No module named 'tsfresh'. 'tsfresh' is a soft dependency and not included in the base sktime installation. Please run: `pip install tsfresh` to install the tsfresh package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)
/usr/local/lib/python3.8/site-packages/sktime/utils/validation/_dependencies.py:143: UserWarning: No module named 'esig'. 'esig' is a soft dependency and not included in the base sktime installation. Please run: `pip install esig` to install the esig package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)


In [4]:
np.sum(y_test == y_pred)/len(y_test)

0.38461538461538464

# Equal Length

## KNeighborsTimeSeriesClassifier

In [5]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score

knn = KNeighborsTimeSeriesClassifier(n_neighbors=5)
param_grid = {"n_neighbors": [1, 5], "distance": ["euclidean", "dtw"]}
parameter_tuning_method = GridSearchCV(knn, param_grid, cv=KFold(n_splits=4))
parameter_tuning_method.fit(X_train, y_train)
y_pred = parameter_tuning_method.predict(X_test)

In [6]:
np.sum(y_test == y_pred)/len(y_test)

0.38461538461538464

## RocketClassifier

In [7]:
from sktime.classification.kernel_based import RocketClassifier

rocket = RocketClassifier(num_kernels=2000)
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)

In [9]:
np.sum(y_test == y_pred)/len(y_test)

0.6153846153846154

## HIVECOTEV2

In [10]:
from sktime.classification.hybrid import HIVECOTEV2

hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)
hc2.fit(X_train, y_train)
y_pred = hc2.predict(X_test)

In [11]:
np.sum(y_test == y_pred)/len(y_test)

0.3076923076923077